# 1. Information Retrieval + Ranking

Sberquad as corpus (db)

\+ easy to implement

\- limited by knowledge base of corpus


In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install rank_bm25

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.10/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.10/dist-packages/spacy/compat.py", line 4, in <module>
    from thinc.util import copy_array
  File "/usr/local/lib/python3.10/dist-packages/thinc/__init__.py", line 5, in <module>
    from .config import registry
  File "/usr/local/lib/python3.10/dist-packages/thinc/config.py", line 5, in <module>
    from .types im

In [ ]:
import spacy

nlp = spacy.load('ru_core_news_sm')

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

question = "Население Москвы составляет 12 миллионов человек. Какое население Москвы?"
preprocessed_question = preprocess_text(question)

In [ ]:
corpus = [
    "Население Москвы составляет 12 миллионов человек.",
    "Париж — столица Франции.",
    "Нью-Йорк — крупнейший город в США."
]

## TF-IDF

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)  # индексация корпуса

def retrieve_best_match(question, vectorizer, X):
    query_vector = vectorizer.transform([question])
    similarities = cosine_similarity(query_vector, X)
    best_match_idx = similarities.argmax()  # Находим индекс самого похожего текста
    return corpus[best_match_idx]

answer = retrieve_best_match(preprocessed_question, vectorizer, X)
print(answer)  # Выведет: "Население Москвы составляет 12 миллионов человек."

Население Москвы составляет 12 миллионов человек.


## BM25

In [ ]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

tokenized_query = preprocessed_question.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
best_doc_idx = doc_scores.argmax()
answer = corpus[best_doc_idx]
print(answer)  # "Население Москвы составляет 12 миллионов человек."


Население Москвы составляет 12 миллионов человек.


# 2. Information Retrieval + RAG

The same as previous, but storing corpus in vector db.

# 3. Encoder-decoder model

Sbequad as train dataset for model

\+ allows model to learn, not to take existing answer from somewhere

\- inference time?

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Загружаем модель и токенайзер T5
model_name = "t5-base"  # Можно использовать "t5-small" или "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Пример вопроса и контекста
input_text = "question: Какое население Москвы? context: Население Москвы составляет 12 миллионов человек."

# Токенизация входного текста
inputs = tokenizer(input_text, return_tensors="pt")

# Генерация ответа
outputs = model.generate(**inputs, max_length=50)

# Декодирование ответа
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Ответ:", answer)

Ответ: 12 миллионов еловек


In [ ]:
from transformers import Trainer, TrainingArguments

# Загружаем данные (например, SberQUAD)
train_data = [...]  # Здесь будет загрузка вашего датасета

# Аргументы для обучения
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_steps=500,
    save_total_limit=2,
)

# Trainer для обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Запуск обучения
trainer.train()


# 4. Start/end token search

\- требуется разделение входного текста на вопрос и контекст

Можно использовать языковые модели для более точного разделения вопроса и контекста, особенно если структура текста неочевидна или разнообразна. Например, вы можете применить модель **NER (Named Entity Recognition)** или классификацию предложений, чтобы автоматически определять, что является вопросом, а что контекстом.

Пример с использованием модели BERT для классификации:
Обучение или использование готовой модели: можно обучить модель, которая будет классифицировать, является ли предложение вопросом или частью контекста. Для этого можно использовать библиотеки, такие как **transformers или spacy**.

Использование NER для идентификации вопросительных предложений: Если у вас есть модели или паттерны, которые распознают вопросительные предложения, вы можете их использовать для разделения.

? что если вопрос и контекст в одном предложении?

In [ ]:
|!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Загрузка предобученной модели и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
question = "Какое население Москвы?"
context = "Население Москвы составляет 12 миллионов человек."

# Токенизация вопроса и контекста
inputs = tokenizer(question, context, return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

# Индексы начального и конечного токенов
start_scores = outputs.start_logits
end_scores = outputs.end_logits

start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

In [ ]:
# Извлечение токенов ответа
tokens = inputs['input_ids'][0][start_index:end_index+1]

# Преобразование токенов обратно в текст
answer = tokenizer.decode(tokens)
print(f"Ответ: {answer}")

Ответ: составляет 12 миллионов человек


# Вывод

1. пробую encoder-decoder:
  - без разделения входного текста
  - с разделением текста на вопрос и контекст

2. start/end token search (с разделением текста на вопрос и контекст)